In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import string
import re
import time as t
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
# The goal of this prototype is to fetch the lyrics of a specific song, in preparation for the full script that will iterate through the yearly data.

songdata = pd.read_csv('data/2019top10songs.csv')

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
cookies = { 'sessionid': '123..'}
i = 0 # Set i to the position in the songdata datafrae you would like to examine.
url = 'https://www.azlyrics.com/lyrics/'
artist = songdata['Artist'][i] # artist1 and artist2 are the most common ways azlyrics represents artist names. artist1 removes spaces while artist2 replaces                                  them with a dash.
artist1 = ''.join(artist.split()).lower().replace('\'','').replace('!','').replace('.','').replace('+','').replace('ñ','n')
artist2 = artist.lower().replace('\'','').replace(' ','-').replace('!','').replace('.','').replace('+','').replace('ñ','n')
artistl = []
if ', ' in artist: # Handles cases where there are three listed artists.
    artistl.append(artist1[:artist1.rfind(',')])
    artistl.append(artist1[artist1.find(',')+1:artist1.rfind('and')])
    artistl.append(artist1[artist1.rfind('and')+3:])
elif ' and ' in artist: # Handles cases where there are two listed artists.
    artistl.append(artist1[:artist1.rfind('and')])
    artistl.append(artist2[:artist2.rfind('-and')])
elif ' featuring ' in artist: # Handles cases where there is a featured artist.
    artistl.append(artist1[:artist1.rfind('featuring')])
    artistl.append(artist2[:artist2.rfind('featuring')])
elif 'bts' in artist1: # Handles that one k-pop group.
    artistl.append('bangtanboys')
else: # Handles cases where there is one artist (which is most of them)
    artistl.append(artist1)
    artistl.append(artist2)
    artist1[artist1.find('the')+3:] # Handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
# The first three entries of the above list are standard single artist titles. The next three handle cases of collaborations between two artists. The next four after that handle collaborations between three artists. The last entry handles a rare case where Wikipedia includes a "the" in the artist name and azlyrics does not.
title = ''.join(songdata['Title'][i].split()).lower().replace('\'','').replace(',','').replace('!','') # azlyrics seems to represent all song titles in the same way.
titlel = [title, 'a' + title, 'the' + title] # However, there are very rare cases, such as 'Holly Jolly Christmas', where azlyrics includes an 'a' or a 'the' at the beginning of a song and Wikipedia does not. These cases will only be tested if all of the artist configurations fail.
success = False
for j in range(len(titlel)):
    for k in range(len(artistl)):
        if requests.get(url + artistl[k] + '/' + titlel[j] + '.html').status_code == 200:
            html = requests.get(url + artistl[k] + '/' + titlel[j] + '.html',cookies=cookies,headers=headers).text
            soup = BeautifulSoup(html, 'lxml')
            print(url + artistl[k] + '/' + titlel[j] + '.html')
            success = True
            t.sleep(5)
            break
        t.sleep(5)
    if success: # Escapes the outer loop when URL succeeds without using a second request.
        break
    if j == len(titlel) - 1:
        print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

https://www.azlyrics.com/lyrics/mariahcarey/alliwantforchristmasisyou.html


In [4]:
artistl

['mariahcarey', 'mariah-carey']

In [5]:
my_text = soup.find('div',attrs={'class': None}).text.replace('\n',' ')[3:] # Extract lyrics text from the azlyrics page and replace newlines with spaces. Also, eliminate the \r tag and space at the beginning.
try:
    my_text = my_text[:my_text.rindex("[")-1] # Remove miscellaneous notes at the end of lyrics.
except ValueError:
    my_text = my_text
my_text = word_tokenize(my_text)
my_text = [i.lower() for i in my_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation.

In [6]:
port_stem = PorterStemmer() # The following code will stem the words in the lyrics. This means that, for example, "stop", "stopping", and "stops" will only count as one unique word.

stem_text = []

for i in range(len(my_text)):
    stem_text.append(port_stem.stem(my_text[i]))

print(stem_text)

['i', 'do', "n't", 'want', 'a', 'lot', 'for', 'christma', 'there', 'is', 'just', 'one', 'thing', 'i', 'need', 'i', 'do', "n't", 'care', 'about', 'the', 'present', 'underneath', 'the', 'christma', 'tree', 'i', 'just', 'want', 'you', 'for', 'my', 'own', 'more', 'than', 'you', 'could', 'ever', 'know', 'make', 'my', 'wish', 'come', 'true', 'all', 'i', 'want', 'for', 'christma', 'is', 'you', 'yeah', 'i', 'do', "n't", 'want', 'a', 'lot', 'for', 'christma', 'there', 'is', 'just', 'one', 'thing', 'i', 'need', 'and', 'i', 'do', "n't", 'care', 'about', 'the', 'present', 'underneath', 'the', 'christma', 'tree', 'i', 'do', "n't", 'need', 'to', 'hang', 'my', 'stock', 'there', 'upon', 'the', 'fireplac', 'santa', 'clau', 'wo', "n't", 'make', 'me', 'happi', 'with', 'a', 'toy', 'on', 'christma', 'day', 'i', 'just', 'want', 'you', 'for', 'my', 'own', 'more', 'than', 'you', 'could', 'ever', 'know', 'make', 'my', 'wish', 'come', 'true', 'all', 'i', 'want', 'for', 'christma', 'is', 'you', 'you', 'babi', 'o

In [7]:
unique_count = len(set(stem_text))
unique_count

110

In [8]:
len(my_text)

329